In [6]:
import os
import json
import shutil

In [10]:
target_json = "/data_ssd/refcoco_g/refcoco_g_paligemma_test.json"
image_root = "/data_ssd/"

In [3]:
wandb_dataset_path = "/data_ssd/USER_DATA/omote/wandb_dataset/refcocog_test"
if not os.path.exists(wandb_dataset_path):
    os.makedirs(wandb_dataset_path)

In [4]:
json_data = json.load(open(target_json, 'r'))
print(len(json_data))

9602


In [5]:
print(json_data[0].keys())

dict_keys(['id', 'image', 'conversations', 'gt_entities_quantized_normalized', 'gt_entities_normalized', 'image_width_height', 'ann_id'])


In [13]:
import tqdm

In [14]:
for d in tqdm.tqdm(json_data):
    img_path = os.path.join(image_root, d['image'])
    shutil.copy(img_path, wandb_dataset_path)

  0%|          | 0/9602 [00:00<?, ?it/s]

100%|██████████| 9602/9602 [00:18<00:00, 510.41it/s]


In [15]:
# make_image_artifact.py
import wandb
from pathlib import Path

ENTITY = "katlab-gifu"
PROJECT = "dataset"
ART_NAME = "refcocog_test"          # 例: 画像アーティファクト名
IMG_DIR = Path(wandb_dataset_path) # ローカルの画像フォルダ

run = wandb.init(entity=ENTITY, project=PROJECT, job_type="build_dataset")

art = wandb.Artifact(name=ART_NAME, type="dataset", description="refcocog_test images")
# ディレクトリごと追加（中のファイルがアップロードされ、W&B 側に保存されます）
art.add_dir(str(IMG_DIR))
run.log_artifact(art)
run.finish()

wandb: Currently logged in as: omote-hideaki-s8 (katlab-gifu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (/data_ssd/USER_DATA/omote/wandb_dataset/refcocog_test)... Done. 3.9s


In [16]:
# make_image_artifact.py
import wandb
from pathlib import Path

ENTITY = "katlab-gifu"
PROJECT = "vis_test"
RUN_NAME = "vis_test_refcocog_test"          # 例: 画像アーティファクト名
IMG_DIR = Path(wandb_dataset_path) # ローカルの画像フォルダ

run = wandb.init(entity=ENTITY, project=PROJECT, name=RUN_NAME)
run.finish()

In [17]:
run = wandb.init(entity=ENTITY, project=PROJECT, id="qbz99n8y", resume="must")

In [18]:
img_art = run.use_artifact("katlab-gifu/dataset/refcocog_test:latest")

In [19]:
img_dir = Path(img_art.download())

wandb: Downloading large artifact refcocog_test:latest, 413.21MB. 2600 files... 
wandb:   2600 of 2600 files downloaded.  
Done. 0:0:8.3 (49.9MB/s)


In [ ]:
CLASS_LABELS = {0: "person", 1: "car", 2: "dog"}


In [ ]:
PREDICTIONS = {
    "img_0001.jpg": [
        (50, 40, 200, 260, 0, 0.91),   # person
        (250, 100, 420, 300, 1, 0.83), # car
    ],
    "img_0002.jpg": [
        (80, 90, 180, 210, 2, 0.77),   # dog
    ],
    # ...
}


In [ ]:

# テーブル作成（画像＋BBox を行ごとに）
table = wandb.Table(columns=["image_id", "image", "n_boxes"])

for image_id, boxes in PREDICTIONS.items():
    # 画像ファイルのパス（Artifact 内の相対パスと合わせる）
    img_path = img_dir / image_id

    # W&B の BBox 仕様に合わせて整形
    box_data = []
    for (minX, minY, maxX, maxY, cls_id, score) in boxes:
        box_data.append({
            "position": { "minX": float(minX), "minY": float(minY),
                          "maxX": float(maxX), "maxY": float(maxY) },
            "class_id": int(cls_id),
            "box_caption": f"{CLASS_LABELS[int(cls_id)]} {score:.2f}",
            "scores": {"confidence": float(score)},
            "domain": "pixel"
        })

    # 画像＋BBox を 1 セルにまとめる（複数BBox/複数クラスOK）
    wb_img = wandb.Image(
        str(img_path),
        boxes={
            "predictions": {
                "box_data": box_data,
                "class_labels": CLASS_LABELS
            }
        }
    )
    table.add_data(image_id, wb_img, len(box_data))

# 1 回の log でテーブルを送る（追記）
wandb.log({"detections": table})



In [20]:
run.finish()